In [1]:
'''Assignment 1: k-NN Classification Algorithm & Application for Letter Recognition
AIT-736, Instructor: Dr. Liao, Date: 9/15/2022
Khanh Nguyen, Anisha Mou, Rohan Jonnakuti, Vishveshwar Kondala

The code below using basic Python functions to build a Condensed k-NN algorithm from scratch using the Hart algorithm 
to implement the basic k-NN classification with a given Letter Recognition dataset for k_number = 1.  
This dataset includes 20,000 examples. 

Here are the steps for Hart Algorithm:
- Step 1: Initialize subset: Randomly select 10% samples from the training set as an initial subset and 90% as a test subset.
- Step 2:   + Classify all remaining samples (test subset) using the initial subset (training set).
            + Identify incorrectly classified samples and transfer to the subset.
- Step 3: Re-run the knn-algorithm with the new training set and test set.              
        
*** The rest of the code is to compute accuracy score and confusion matrix with graph (similar to problem 1.1)
'''
import math
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from collections import Counter
# %matplotlib inline
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

#Changing path to working directory
working_directory = os.getcwd()
os.chdir(working_directory)

In [2]:
%%time
#Split the Letter Recognition Dataset
df = pd.read_csv('letter-recognition_data1.csv')

#shuffling the dataset randomly
# df = df.sample(frac = 1)

def split_dataframe_by_position(df, splits):
    """
Randomly select 10% samples from the training set as an initial subset 
and 90% as a test subset
    """
    dataframes = []
    index_to_split = 17999
    start = 0
    end = index_to_split
    for split in range(splits):
        temporary_df = df.iloc[start:end, :]
        dataframes.append(temporary_df)
        start += index_to_split
        end += index_to_split
    return dataframes
split_dataframes = split_dataframe_by_position(df, 2)

trainset = split_dataframes[1]
testset1 = split_dataframes[0]

def split_dataframe_by_position1(df, splits):
    """
Randomly select 10% samples from the training set as an initial subset 
and 90% as a test subset
    """
    dataframes = []
    index_to_split = 1000
    start = 0
    end = index_to_split
    for split in range(splits):
        temporary_df = df.iloc[start:end, :]
        dataframes.append(temporary_df)
        start += index_to_split
        end += index_to_split
    return dataframes

split_dataframes1 = split_dataframe_by_position1(testset1, 2)
testset = split_dataframes1[0]

# trainset.to_csv('trainset.csv')
# testset.to_csv('testset.csv')
print('Randomly select 10% samples from the training set as an initial subset and 90% as a test subset')
print("Trainset size:", trainset.shape)
print("testset size:", testset.shape)


Randomly select 10% samples from the training set as an initial subset and 90% as a test subset
Trainset size: (2000, 17)
testset size: (1000, 17)
Wall time: 39 ms


In [3]:
''' First step of HART Algorithm:  initialize a subset for training and testing'''
# Convert training and testing set to lists
training = trainset.values.tolist()
testing = testset.values.tolist()
print("training set:", training[:10])

# Create a list exclude the labels (the column with letters) to test the algorithm
test1 = testset.iloc[:,:-1] #all data except last column
test1 = test1.values.tolist()
print("Test1 sample: ", test1[:10])
#Extract the actual letters from the test set (last column) and convert to a list for Accuracy calculation
test_letters = testset.iloc[:, -1:]  #select only the last column
test_letters = test_letters.values.tolist()
actual_letters = list()
for letter in test_letters:
    actual_letters += letter
# print(actual_letters)

#Create the test_subset by joining test1 and its original labels (column) for comparison with predicted results
test_subset =[]
for a, b in zip(test1, actual_letters):
    test_subset.append([a,b])
    
print("test_subset is:", test_subset[:5])

training set: [[5, 8, 6, 10, 9, 8, 9, 3, 2, 6, 8, 7, 6, 10, 6, 4, 'P'], [3, 5, 5, 3, 4, 7, 6, 3, 4, 9, 7, 8, 7, 6, 1, 8, 'M'], [8, 11, 7, 8, 5, 3, 11, 1, 3, 8, 10, 7, 6, 11, 2, 7, 'V'], [4, 5, 5, 4, 2, 4, 12, 3, 6, 12, 10, 4, 1, 11, 2, 5, 'Y'], [3, 4, 4, 5, 1, 7, 7, 4, 4, 7, 6, 8, 3, 8, 4, 8, 'X'], [1, 0, 2, 1, 0, 8, 9, 3, 2, 6, 12, 8, 2, 10, 0, 8, 'V'], [2, 5, 4, 4, 2, 10, 6, 2, 6, 12, 4, 9, 1, 6, 1, 7, 'J'], [2, 3, 2, 2, 1, 7, 7, 6, 3, 8, 6, 8, 2, 8, 2, 7, 'O'], [5, 6, 7, 4, 5, 9, 6, 2, 4, 9, 6, 7, 7, 6, 2, 7, 'M'], [5, 5, 6, 4, 2, 4, 12, 3, 3, 10, 11, 7, 2, 10, 1, 8, 'V']]
Test1 sample:  [[5, 12, 3, 7, 2, 10, 5, 5, 4, 13, 3, 9, 2, 8, 4, 10], [4, 11, 6, 8, 6, 10, 6, 2, 6, 10, 3, 7, 3, 7, 3, 9], [7, 11, 6, 6, 3, 5, 9, 4, 6, 4, 4, 10, 6, 10, 2, 8], [2, 1, 3, 1, 1, 8, 6, 6, 6, 6, 5, 9, 1, 7, 5, 10], [4, 11, 5, 8, 3, 8, 8, 6, 9, 5, 6, 6, 0, 8, 9, 7], [4, 2, 5, 4, 4, 8, 7, 6, 6, 7, 6, 6, 2, 8, 7, 10], [1, 1, 3, 2, 1, 8, 2, 2, 2, 8, 2, 8, 1, 6, 2, 7], [2, 2, 4, 4, 2, 10, 6, 2, 6, 12, 4, 8,

In [4]:
%%time
# Make Predictions with k-nearest neighbors on the Letter Recognition Dataset

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
		print('[%s] => %d' % (value, i))
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

            # Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
# 	return neighbors.pd.DataFrame()
	return neighbors


Wall time: 0 ns


In [5]:
%%time
"""STEP 2 of HART algorithm: Classify all remaining samples using the subset"""

# Make the first classification prediction with the initial training subset
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return [test_row, prediction]

#Use k number of neighbors and predict the letters with the Euclidean-distance algorithms above
num_neighbors = 1
# predict the label
predicted = list()
for row in test1:
    label = predict_classification(training, row, num_neighbors)
    predicted.append(label)

# Calculate accuracy percentage
def accuracy_metric(actual, predicted): 
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

#get the Accuracy score for the nearest neighbor algorithm
accuracy_score = accuracy_metric(test_subset, predicted)
print("The accuracy score of the KNN model initial subset is:", accuracy_score)
print("The first classification of all remaining samples: ", predicted[:10])

The accuracy score of the KNN model initial subset is: 81.89999999999999
The first classification of all remaining samples:  [[[5, 12, 3, 7, 2, 10, 5, 5, 4, 13, 3, 9, 2, 8, 4, 10], 'I'], [[4, 11, 6, 8, 6, 10, 6, 2, 6, 10, 3, 7, 3, 7, 3, 9], 'B'], [[7, 11, 6, 6, 3, 5, 9, 4, 6, 4, 4, 10, 6, 10, 2, 8], 'N'], [[2, 1, 3, 1, 1, 8, 6, 6, 6, 6, 5, 9, 1, 7, 5, 10], 'G'], [[4, 11, 5, 8, 3, 8, 8, 6, 9, 5, 6, 6, 0, 8, 9, 7], 'S'], [[4, 2, 5, 4, 4, 8, 7, 6, 6, 7, 6, 6, 2, 8, 7, 10], 'B'], [[1, 1, 3, 2, 1, 8, 2, 2, 2, 8, 2, 8, 1, 6, 2, 7], 'A'], [[2, 2, 4, 4, 2, 10, 6, 2, 6, 12, 4, 8, 1, 6, 1, 7], 'J'], [[11, 15, 13, 9, 7, 13, 2, 6, 2, 12, 1, 9, 8, 1, 1, 8], 'M'], [[3, 9, 5, 7, 4, 8, 7, 3, 8, 5, 6, 8, 2, 8, 6, 7], 'X']]
Wall time: 15.4 s


In [6]:
%%time
""""Step2 and Step3 of HART algorithm: Extracting all the incorrect labels and transfer to the subset."""
def extract_incorrect_label(actual, predicted): 
    correct = 0
    incorrect_labels = list()
    removed_incorrects = list() 
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
        else: 
            incorrect_labels.append(predicted[i])
            removed_incorrects.append(actual[i])
    return incorrect_labels, removed_incorrects

incorrect_labels, removed_incorrects = extract_incorrect_label(test_subset, predicted)
print("incorrect classifieds to be removed and transferred:", len(removed_incorrects))
print("incorrect labels to be added to the initial subset:", len(incorrect_labels))


incorrect classifieds to be removed and transferred: 181
incorrect labels to be added to the initial subset: 181
Wall time: 4 ms


In [7]:
%%time
"""FIRST RUN OF STEP 2 followed HART algorithm: Transfer (remove) incorrect classified labels from test subset"""
new_test_subset = test_subset
for element in removed_incorrects:
    if element in test_subset:
        new_test_subset.remove(element)
print(len(new_test_subset))
new_test_subset[:4]

819
Wall time: 8 ms


[[[5, 12, 3, 7, 2, 10, 5, 5, 4, 13, 3, 9, 2, 8, 4, 10], 'I'],
 [[7, 11, 6, 6, 3, 5, 9, 4, 6, 4, 4, 10, 6, 10, 2, 8], 'N'],
 [[2, 1, 3, 1, 1, 8, 6, 6, 6, 6, 5, 9, 1, 7, 5, 10], 'G'],
 [[4, 11, 5, 8, 3, 8, 8, 6, 9, 5, 6, 6, 0, 8, 9, 7], 'S']]

In [8]:
new_test_subset_df = pd.DataFrame(new_test_subset)
# print(new_test_subset_df.head())
non_labels_df = new_test_subset_df.iloc[:,:-1] #all data except last column
labels_only_df = new_test_subset_df.iloc[:,-1:]

# labels_new_array = labels_only_df.to_numpy()
print(non_labels_df.head())
print(labels_only_df.head())
# non_labels_df.to_csv('new_test_df.csv', index=False)

                                                   0
0  [5, 12, 3, 7, 2, 10, 5, 5, 4, 13, 3, 9, 2, 8, ...
1  [7, 11, 6, 6, 3, 5, 9, 4, 6, 4, 4, 10, 6, 10, ...
2  [2, 1, 3, 1, 1, 8, 6, 6, 6, 6, 5, 9, 1, 7, 5, 10]
3  [4, 11, 5, 8, 3, 8, 8, 6, 9, 5, 6, 6, 0, 8, 9, 7]
4  [4, 2, 5, 4, 4, 8, 7, 6, 6, 7, 6, 6, 2, 8, 7, 10]
   1
0  I
1  N
2  G
3  S
4  B


In [9]:
"""Apply the new test dataset after transferred out the incorrect classified samples"""
new_test = pd.read_csv('new_test_df.csv')
new_test_df = new_test.values.tolist()
new_test_df[:4]

print("The count of new testing samples after transferred the incorrect classified samples: ",len(new_test_df))

test2 = new_test.iloc[:,:-1] #all data except last column
test2 = new_test.values.tolist()
# print("Test1 sample: ", test1[:10])
#Extract the actual letters from the test set (last column) and convert to a list for Accuracy calculation
test_letters1 = new_test_subset_df.iloc[:, -1:]  #select only the last column
test_letters1 = test_letters1.values.tolist()
actual_letters1 = list()
for letter in test_letters1:
    actual_letters1 += letter

actual_letters2 = actual_letters1[1:] 
print(len(actual_letters2))
print(actual_letters[:5])
#Create the test_subset by joining test1 and its original labels (column) for comparison with predicted results
test_subset1 = []
for a, b in zip(test2, actual_letters1):
    test_subset.append([a,b])
    
# print("test_subset1 is:", test_subset1[:5])
# print(test2[:4])


FileNotFoundError: [Errno 2] No such file or directory: 'new_test_df.csv'

In [ ]:
incorrect_df = pd.DataFrame(incorrect_labels)
print("The total number of incorrect classified samples:", len(incorrect_df))
# incorrect_df.to_csv('incorrect_label.csv', index=False)
# Manipulate the csv file to correct format for training subset.


In [ ]:
'''Step2 Hart Algorithm:  Transferred incorrect classified samples to the subset: new_training_subset'''
new_train_df = pd.read_csv('incorrect_label.csv')
new_train_subset = new_train_df.values.tolist()

# new_train_subset = training
new_training_subset = training + new_train_subset
print("The count of new training subset after transferred the incorrect classified samples: ",len(new_training_subset))

In [ ]:
%%time
""" REPEATED STEP 2 of HART algorithm: Classify all remaining samples using the subset"""

# Make the first classification prediction with the initial training subset
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

#Use k number of neighbors and predict the letters with the Euclidean-distance algorithms above
num_neighbors = 1
# predict the label
predicted1 = list()
for row in new_test_df:
    label = predict_classification(new_training_subset, row, num_neighbors)
    predicted1.append(label)

# Calculate accuracy percentage
def accuracy_metric(actual, predicted): 
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

#get the Accuracy score for the nearest neighbor algorithm
accuracy_score = accuracy_metric(actual_letters2, predicted1)
print("The accuracy score of the Condensed Nearest Neighbor model after applying HART algorithm to the subset is:", accuracy_score)
print("The final classification of all remaining samples : ", predicted1[:10], "and the count:", len(predicted1))

In [ ]:
# Calculate Confusion Matrix without scikit-learn
def confusionmatrix(actual, predicted, normalize = False):
    """
    Generate a confusion matrix for multiple classification
    @params:
        actual      - a list of integers or strings for known classes
        predicted   - a list of integers or strings for predicted classes
        normalize   - optional boolean for matrix normalization
    @return:
        matrix      - a 2-dimensional list of pairwise counts
    """
    unique = sorted(set(actual))
    matrix = [[0 for _ in unique] for _ in unique]
    imap   = {key: i for i, key in enumerate(unique)}
    # Generate Confusion Matrix
    for p, a in zip(predicted, actual):
        matrix[imap[p]][imap[a]] += 1
    # Matrix Normalization
    if normalize:
        sigma = sum([sum(matrix[imap[i]]) for i in unique])
        matrix = [row for row in map(lambda i: list(map(lambda j: j / sigma, i)), matrix)]
    return matrix

cm = confusionmatrix(actual_letters2, predicted1)
print("Confusion matrix for each letter: A-Z")
for line in cm:
    print(line)

In [ ]:
#Plotting the confusion matrix

plt.figure(figsize=(20, 14))
sns.heatmap(cm, annot=True)
plt.rc('axes.formatter', useoffset=False)
plt.title('Confusion Matrix', size = 25)
plt.ylabel('Actual Values', size = 25)
plt.xlabel('Predicted Values', size = 25)
# plt.gca().yaxis.set_major_formatter(plt.ScalarFormatter(useMathText=False))
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.show()


## INTERPRETATION:
The accuracy score of the Condensed Nearest Neighbor built from scratch is at aroun 81%, which is much lower than Task 2.1 at 94% but it is based on smaller training size.  Also the training subset was diluted by incorrect classified samples due to following the Hart Algorithm from the Lecture slide.  

## REFERENCE
Liao, L. (2022). Supervised Learning KNN algorithms.  George Mason University. Applied Machine Learning Course.